# Лабораторная работа №3

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
import os
os.chdir("C:/Users/вячеслав/Desktop/Lab_01/Lab_01/")

In [3]:
df = pd.read_csv('Winequality-red.csv', sep=';')

In [4]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
# смотрим на размерность матрицы данных
df.shape

(1599, 12)

In [6]:
# смотрим сбалансированы ли классы, они не сбалансированы 
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [7]:
df['quality'] = df['quality'].replace(to_replace=3, value=4).replace(8, 7)

In [8]:
df['quality'].value_counts()

5    681
6    638
7    217
4     63
Name: quality, dtype: int64

In [27]:
#  разделяю предикоторы и отклики
y = df['quality']
X = df.drop(columns=['quality'])

In [28]:
#  разделяю на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 1234) 

In [29]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [30]:
param_grid = {  
    'n_estimators': [100, 300, 500, 800, 1000, 1500],
    'max_features': ['sqrt', 5, 7, 9, 10],
    'max_depth': [3, 4, 6, 8, 10],
}

In [31]:
model = GradientBoostingClassifier(random_state=1234,
                                   # Доля наблюдений в случайной подвыборке для очередного дерева
                                   subsample=0.66,
                                   # Доля переменных в случайной подвыборке для очередного дерева
                                   max_features='sqrt', 
                                   # Число деревьев
                                   n_estimators=500,
                                   #  критерий качества ‘deviance’ (кросс-энтропия) или ‘exponential’ (как в AdaBoost)
                                   #  ‘deviance’ для классификации с вероятностями на выходе
                                   loss='deviance', 
                                   # shrinkage На это число умножаем каждое дерево.
                                   # Рекомендуется выставлять небольшие значения из (0, 0.3].
                                   learning_rate=0.01, 
                                   #  загрязнение дерева измеряем “mse” или “friedman_mse”  (mse с улучшениями)
                                    #  “mae” удалено из нынешней версии,  
                                   criterion='friedman_mse', 
                                   # минимальное уменьшение загрязнения 
                                   min_impurity_decrease=0.001, 
                                   # минимальное число наблюдений в узле потомке
                                   min_samples_leaf=5, 
                                   # минимальное число наблюдений в узле родителе
                                   min_samples_split=10,
                                   # число узлов в дереве (в RandomForest иначе !!!!!!!!!!!!!)
                                   max_depth=5,
                                   # как выдавать промежуточные результаты
                                   verbose=0
                                   # Есть и другие параметры, уменьшающие размер дерева,
                                   # такие же как у DecisionTreeClassifier
                                   )
grid_search_res = GridSearchCV(model, param_grid=param_grid, cv=2, error_score='raise', 
                               pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [32]:
grid_search_res.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.01,
                                                  loss='deviance', max_depth=5,
                                                  max_features='sqrt',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.001,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=5,
                                                  min_samples_split=10,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=500,
                                                  n_iter_no_change=None,
                                                  presort='auto'

In [33]:
print("Best params on dev set:")
print(grid_search_res.best_params_)

Best params on dev set:
{'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 500}


In [34]:
# строю предсказание модели
y_pred_train = grid_search_res.predict(X_train)
y_pred_test = grid_search_res.predict(X_test)

In [35]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,4,5,6,7
4,1,10,4,1
5,1,183,45,2
6,0,49,134,34
7,0,1,27,36


In [36]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,4,5,6,7
4,46,1,0,0
5,0,444,6,0
6,0,8,413,0
7,0,1,3,149


In [37]:
# строим предсказание модели
y_pred_train2 = grid_search_res.predict_proba(X_train)
y_pred_test2 = grid_search_res.predict_proba(X_test)
print(y_pred_train2)
print(y_pred_test2)

[[4.39752596e-04 1.71969602e-02 4.38595137e-02 9.38503773e-01]
 [1.83262310e-03 2.98200087e-01 6.94478647e-01 5.48864327e-03]
 [5.08558209e-03 9.73992158e-01 1.86661718e-02 2.25608772e-03]
 ...
 [9.66222757e-03 2.92226305e-01 6.90739045e-01 7.37242284e-03]
 [3.56910856e-03 8.35666777e-01 1.27897492e-01 3.28666224e-02]
 [2.96411440e-03 6.30445970e-01 2.96215031e-01 7.03748847e-02]]
[[0.106567   0.69280188 0.18740339 0.01322773]
 [0.00305713 0.70942576 0.26936781 0.0181493 ]
 [0.00576001 0.15425595 0.69592596 0.14405809]
 ...
 [0.01735264 0.10921451 0.6575783  0.21585454]
 [0.00122577 0.03797181 0.2462076  0.71459482]
 [0.00689143 0.62900138 0.35721222 0.00689497]]


In [38]:
y_pred_train2 = grid_search_res.predict(X_train)
y_pred_test2 = grid_search_res.predict(X_test)
print(metrics.classification_report(y_pred_test2, y_test))

              precision    recall  f1-score   support

           4       0.06      0.50      0.11         2
           5       0.79      0.75      0.77       243
           6       0.62      0.64      0.63       210
           7       0.56      0.49      0.53        73

    accuracy                           0.67       528
   macro avg       0.51      0.60      0.51       528
weighted avg       0.69      0.67      0.68       528



In [39]:
fi = pd.DataFrame({'features': X_train.columns, 'importance': grid_search_res.best_estimator_.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)

,features,importance
10,alcohol,0.154597
9,sulphates,0.122906
1,volatile acidity,0.111767
6,total sulfur dioxide,0.105151
7,density,0.095294
4,chlorides,0.076896
8,pH,0.071822
2,citric acid,0.071426
0,fixed acidity,0.071324
3,residual sugar,0.060569


In [40]:
# оцениваем долю наблюдений в тестовой выборке, для которых есть класс,вероятность принадлежать которому больше 0.8
# до калибровки
y_pred_train2 = grid_search_res.predict_proba(X_train)
y_pred_test2 = grid_search_res.predict_proba(X_test)
k = 0
for i in y_pred_test2:
    for j in i:
        if (j>0.8):
            k += 1
dolya_do = k/len(y_pred_test2)*100

In [41]:
dolya_do

30.303030303030305

# Делаем калибровку

In [42]:
from sklearn.calibration import CalibratedClassifierCV

In [43]:
model_sigmoid = CalibratedClassifierCV(grid_search_res, cv=2, method='sigmoid')

In [ ]:
model_sigmoid.fit(X_train, y_train)

In [ ]:
y_pred_train3 = model_sigmoid.predict(X_train)
y_pred_test3 = model_sigmoid.predict(X_test)
print(metrics.classification_report(y_pred_test3, y_test))

In [ ]:
y_pred_train3 = model_sigmoid.predict_proba(X_train)
y_pred_test3 = model_sigmoid.predict_proba(X_test)
print(y_pred_train3)
print(y_pred_test3)

In [ ]:
# оцениваем долю наблюдений в тестовой выборке, для которых есть класс,вероятность принадлежать которому больше 0.8
# после калибровки
k = 0
for i in y_pred_test3:
    for j in i:
        if (j>0.8):
            k += 1
dolya_posle = k/len(y_pred_test3)*100

In [ ]:
dolya_posle